In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from datetime import datetime
import io
import os
import sys

from IPython.display import Video as JupyterVideo
from IPython.display import HTML
from IPython.display import display
from os import system, name

os.environ['PYOPENGL_PLATFORM'] = 'osmesa'

import pose_pipeline
from pose_pipeline import *
from pose_pipeline.utils.jupyter import play, play_grid
pose_pipeline.set_environmental_variables()


[Data organization](erd.png)

## Import test videos

In [ ]:
from pose_pipeline.utils.video_format import insert_local_video

test_videos_path = '/videos'
files = os.listdir(test_videos_path)

for f in files:
    insert_local_video(f, datetime.now(), os.path.join(test_videos_path, f), video_project='DEMO_TEST', skip_duplicates=True)

In [ ]:
filt = {'video_project': "DEMO_TEST"}

# List videos in database
VideoInfo.populate(filt)
Video & 'video_project="DEMO_TEST"'

In [ ]:
from pose_pipeline.pipeline import BottomUpBridging,BottomUpBridgingPerson

BottomUpBridging.populate(filt)

In [ ]:
BottomUpBridging()

In [ ]:
video_keys = (Video & 'video_project="DEMO_TEST"').fetch('KEY')

for v in video_keys:
    print(v)
    v["bottom_up_method_name"] = "Bridging_OpenPose"

    BottomUpMethod.insert1(v, skip_duplicates=True)

In [ ]:
BottomUpPeople.populate(filt)

In [ ]:
BottomUpPeople()

In [ ]:
BlurredVideo.populate()

In [ ]:
BlurredVideo()

## Run tracking algorithm

After running this, use the GUI to annotate the video to indicate the subject of interest.

In [ ]:
# select tracking method

filt = 'video_project="DEMO_TEST"'

keys = ((Video & filt)).fetch('KEY')

print(f'Missing tracking methods: {len(keys)}')
tracking_method = (TrackingBboxMethodLookup & 'tracking_method_name="MMDet_qdtrack"').fetch1('tracking_method')
for key in keys:
    key['tracking_method'] = tracking_method
    TrackingBboxMethod.insert1(key, skip_duplicates=True)

In [ ]:
TrackingBboxMethod()

In [ ]:
TrackingBbox()

In [ ]:
# OpenPose.populate(filt)
TrackingBbox.populate(filt)

In [ ]:
TrackingBbox()

In [ ]:
BottomUpPeople & video_keys & 'bottom_up_method_name="Bridging_OpenPose"'

In [ ]:
# produce visualizations of these outputs used for annotation

BlurredVideo.populate(filt)
TrackingBboxVideo.populate(filt)

In [ ]:
play(TrackingBboxVideo() & {'tracking_method':9} & {'filename':'koa_212_test.mp4'})

In [ ]:
# Run annotation GUI
to_label = ((TrackingBboxVideo & filt) - PersonBboxValid).fetch('KEY')

def assign_video(labeling_key, valid, tracks=None):
    """
    Params:
        labeling_key: dictionary indicating the video being visualized to annotate
        valid: boolean indicating a valid person could be identified
        tracks: list matching the numbers associated with the person in video
    """
    
    # video_subject_id = 0 corresponds to a valid subject, and
    # video_subject_id = -1 corresponds to a video with bad person detection
    # the keep_tracks fields should correspond to the number(s) overlying the
    # person in the video

    video_key = (TrackingBbox & labeling_key).fetch1('KEY')

    key = video_key.copy()
    if valid:
        key['video_subject_id'] = 0
        key['keep_tracks'] = tracks
    else:
        key['video_subject_id'] = -1
        key['keep_tracks'] = []
        
    print(f'Inserting {key} into database') #Inserts into database 
    PersonBboxValid.insert1(key)
    
def unlabel(labeling_key):
    (PersonBboxValid & labeling_key).delete()


def show_annotation(idx):
    
    labeling_key = to_label[idx]
    print(f'Showing: {labeling_key}')
    
    video = (TrackingBboxVideo & labeling_key).fetch1('output_video')
    
    # get the track IDs present in the video
    tracks = (TrackingBbox & labeling_key).fetch1('tracks')
    tracks = np.unique([[t['track_id'] for track in tracks for t in track]]).tolist()
    
    display(JupyterVideo(video, height=480, html_attributes="controls muted autoplay"))
    practice = widgets.ToggleButtons(
        options= ['Default', *tracks, 'Multiple', 'Skip','Undo', 'Invalid'],
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Can't Identify Subject", '0', '1', '2', '3', 'Skip']
    )   
    display(practice)
    
    multiple = widgets.Text(
        value='',
        placeholder='',
        description='Multiple',
        disabled=False
    )
    display(multiple)

    def on_click(change):
        value = change['new']
        if value == 'Multiple':
            value = [int(v) for v in multiple.value.split(',')]
            assign_video(labeling_key, True, value)
        elif value == 'Skip':
            print('Skipping')
        elif value == 'Invalid':
            print('Flagging Invalid')
            assign_video(labeling_key, False)
        elif value == 'Undo':
            unlabel(to_label[idx-1])
            show_annotation(idx-1)
            return
        else:
            value = int(value)
            assign_video(labeling_key, True, [value])
       
        os.remove(video)
        
        show_annotation(idx + 1)

    practice.observe(on_click, 'value'"")
    ""
show_annotation(0)


## Perform additional analysis on the selected subject

In [ ]:
# compute the final bounding box for subjects of interest
pose_pipeline.PersonBbox.populate(filt)

# select the top down approach to run
top_down_method = (pose_pipeline.TopDownMethodLookup & 'top_down_method_name="MMPose"').fetch1('top_down_method')
keys = ((pose_pipeline.PersonBbox & filt) - (pose_pipeline.TopDownMethod & {'smpl_method': smpl_method})).fetch('KEY')
print(f'Missing top down methods: {len(keys)}')
for key in keys:
    key['top_down_method'] = top_down_method
    pose_pipeline.TopDownMethod.insert1(key)

# select an SMPL method to run
smpl_method = (pose_pipeline.SMPLMethodLookup & 'smpl_method_name="PARE"').fetch1('smpl_method')
keys = ((pose_pipeline.PersonBbox & filt) - (pose_pipeline.SMPLMethod & {'smpl_method': smpl_method})).fetch('KEY')
print(f'Missing SMPL methods: {len(keys)}')
for key in keys:
    key['smpl_method'] = smpl_method
    pose_pipeline.SMPLMethod.insert1(key)

In [ ]:
# populate the selected analyses

TopDownPerson.populate(filt)
SMPLPerson.populate(filt)

## Produce and review visualizations

In [ ]:
TopDownPersonVideo.populate(filt)
SMPLPersonVideo.populate(filt)

In [ ]:
keys = (TopDownPersonVideo & filt).fetch('KEY')
play(TopDownPersonVideo & keys[0])